Найдите датасет 'titanic.csv', где содержатся сведения о пассажирах «Титаника». Доработайте код отбора признаков, использовав модели линейной регрессии (LinearRegression) и гребневой регрессии (Ridge) Отобразите получившиеся значения\оценки каждого признака каждым методом\моделью и среднюю оценку. Проведите анализ получившихся результатов. Какие 4 признака оказались самыми важными по среднему значению?

**Датасет был взят с сайта kaggle и загружен на личный GitHub**

#Импорт библиотек

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#Загрузка датасета

In [27]:
# Загрузка датасета с GitHub
url = 'https://raw.githubusercontent.com/happy9/Titanic/main/titanic.csv'
df = pd.read_csv(url)

In [28]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


#Предобработка данных

Pclass — Класс пассажира<br>
Sex — Пол пассажира<br>
Age — Возраст пассажира<br>
SibSp — Количество родственников на борту<br>
Parch — Количество родителей/детей на борту<br>
Fare — Стоимость билета<br>
Embarked — Порт посадки

In [29]:
# Предобработка данных
# Выбираем релевантные признаки
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

# Преобразуем категориальные переменные в dummy/индикаторные переменные
X = pd.get_dummies(X, drop_first=True)

# Обрабатываем пропущенные значения
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Стандартизируем признаки
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

#Разделение данных

In [30]:
# Разделяем данные на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#Обучение моделей

In [31]:
# Обучаем модель линейной регрессии
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Обучаем модель гребневой регрессии
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

Ridge()

#Вычисление важности признаков

In [32]:
# Получаем важность признаков (коэффициенты) для обеих моделей
lr_importance = lr_model.coef_
ridge_importance = ridge_model.coef_

# Объединяем результаты в DataFrame
feature_names = X.columns
importance_df = pd.DataFrame({
    'Признак': feature_names,
    'Линейная регрессия': lr_importance,
    'Гребневая регрессия': ridge_importance
})

# Вычисляем среднюю важность
importance_df['Средняя важность'] = importance_df[['Линейная регрессия', 'Гребневая регрессия']].mean(axis=1)

# Сортируем признаки по средней важности
importance_df_sorted = importance_df.sort_values(by='Средняя важность', ascending=True)

#Вывод результатов

In [33]:
# Выводим отсортированный DataFrame с важностью признаков
print(importance_df)

# Выводим 4 самых важных признака по средней важности
top_4_features = importance_df_sorted.head(4)
print("\nТоп-4 самых важных признака по средней важности:")
print(top_4_features)

      Признак  Линейная регрессия  Гребневая регрессия  Средняя важность
0      Pclass           -0.128944            -0.128682         -0.128813
1         Age           -0.062439            -0.062281         -0.062360
2       SibSp           -0.042671            -0.042626         -0.042649
3       Parch           -0.015980            -0.015893         -0.015937
4        Fare            0.017336             0.017470          0.017403
5    Sex_male           -0.244895            -0.244545         -0.244720
6  Embarked_Q           -0.006528            -0.006521         -0.006524
7  Embarked_S           -0.033296            -0.033284         -0.033290

Топ-4 самых важных признака по средней важности:
    Признак  Линейная регрессия  Гребневая регрессия  Средняя важность
5  Sex_male           -0.244895            -0.244545         -0.244720
0    Pclass           -0.128944            -0.128682         -0.128813
1       Age           -0.062439            -0.062281         -0.062360
2     Sib

#Вывод

На основании полученных данных можно сделать следующие выводы:<br>
<br>
Самыми важными признаками по среднему значению важности оказались:<br>
1. Sex_male<br>
2. Pclass<br>
3. Age<br>
4. SibSp<br>

Эти признаки имеют наибольшее абсолютное значение коэффициентов, что указывает на их значительное влияние на вероятность выживания пассажиров.

Признак Sex_male имеет наибольший отрицательный коэффициент, что означает, что будучи мужчиной, вероятность выживания снижалась.

Признак Pclass также имеет значительное отрицательное значение, что указывает на меньшую вероятность выживания пассажиров из низших классов.

Признаки Age и SibSp также оказывают влияние, но в меньшей степени.

Эти результаты соответствуют известным фактам о катастрофе Титаника, где женщины и пассажиры высших классов имели больше шансов на выживание.